In [1]:
from PIL import Image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dropout, LeakyReLU, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.callbacks import EarlyStopping
from keras.losses import binary_crossentropy
from tensorflow.keras import regularizers
import tensorflow as tf

In [2]:
height, width, channels = 376, 1248, 1  #input image format
input_img = Input(shape=(height, width, channels))
#print(input_img)

In [3]:
# Encoder
x = Conv2D(32, (3, 3), padding='same')(input_img)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Dropout(0.2)(x)
x = Conv2D(16, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Dropout(0.2)(x)
x = Conv2D(8, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.1)(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)
# Decoder
x = Conv2D(8, (3, 3), padding='same')(encoded)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.1)(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.1)(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.1)(x)
x = UpSampling2D((2, 2))(x)

decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

/Users/yong/anaconda3/envs/plard/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [4]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer=Adam(learning_rate = 0.001), loss='mse')
autoencoder.summary()

def preprocess_image(image):
    #resize image so they're same format
    image = tf.image.resize(image, (height, width), method='lanczos5')
    return image

datagen = ImageDataGenerator(
    #normalize image so values are 0>= and <=1
    rescale=1./255,
    preprocessing_function=preprocess_image,
    validation_split=0.2
)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 376, 1248, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 376, 1248, 32)  │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 376, 1248, 32)  │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 376, 1248, 32)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 188, 624, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 188, 624, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 188, 624, 16)   │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 188, 624, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 188, 624, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 94, 312, 16)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 94, 312, 16)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 94, 312, 8)     │         1,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 94, 312, 8)     │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 94, 312, 8)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 47, 156, 8)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 47, 156, 8)     │           584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 47, 156, 8)     │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 47, 156, 8)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 94, 312, 8)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 94, 312, 16)    │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 94, 312, 16)    │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 94, 312, 16)    │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 13,233 (51.69 KB)

 Trainable params: 13,009 (50.82 KB)

 Non-trainable params: 224 (896.00 B)

In [5]:
import os
os.getcwd()

'/Users/yong/Desktop'

In [5]:
train_generator = datagen.flow_from_directory(
    'thesis/dataset',  #path to dataset
    target_size=(height, width),
    color_mode='grayscale',
    batch_size=16,
    class_mode='input',
    subset='training'
)
validation_generator = datagen.flow_from_directory(
    'thesis/dataset',  #path to dataset
    target_size=(height, width),
    color_mode='grayscale',
    batch_size=16,
    class_mode='input',
    subset='validation'
)

Found 66 images belonging to 5 classes.
Found 14 images belonging to 5 classes.


In [6]:
# Define early stopping callback
early_stopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Now include the callback in the fit method
autoencoder.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[early_stopper]
)

Epoch 1/10


/Users/yong/anaconda3/envs/plard/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 40s 7s/step - loss: 0.2184 - val_loss: 0.2344
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 37s 7s/step - loss: 0.0531 - val_loss: 0.2183
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 36s 7s/step - loss: 0.0211 - val_loss: 0.1910
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 34s 6s/step - loss: 0.0094 - val_loss: 0.1640
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 36s 7s/step - loss: 0.0058 - val_loss: 0.1391
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 39s 9s/step - loss: 0.0046 - val_loss: 0.1171
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 39s 9s/step - loss: 0.0036 - val_loss: 0.0997
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 41s 8s/step - loss: 0.0032 - val_loss: 0.0861
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 39s 7s/step - loss: 0.0024 - val_loss: 0.0769
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 39s 7s/step - loss: 0.0022 - val_loss: 0.0713
Restoring model weights from the end of the best epoch: 10.


In [7]:
#save model
autoencoder.save('0428_2.keras')